# Lab 5 Data Structures

Skills
- reading xml
- reading json

Extra Modules
- json

Resources
- https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.io.json.json_normalize.html
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html
- https://docs.python.org/2/library/xml.etree.elementtree.html

In [2]:
# import modules
import pandas as pd
import numpy as np

from pandas.io.json import json_normalize #special package in pandas
import json
from lxml import etree
import sqlite3

# Part A: Reading json

In [122]:
# example JSON
players = '[{"name":"Gazinsky","team":"Russia"},{"name":"Dzyuba","team":"Russia"},{"name":"Lukaku","team":"Belgium"}]'
stadium_data = '{"stadiums":[{"name": "Ekaterinburg Arena","city": "Ekaterinburg"},{"name": "Luzhniki Stadium","city": "Moscow"},{"name": "Nizhny Novgorod Stadium","city": "Nizhny Novgorod"}]}'



In [123]:
# load players string as a json object
playersobj = json.loads(players)
# print the dictionary containing information about the second player only
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
print(playersobj[1])

{'name': 'Dzyuba', 'team': 'Russia'}


In [124]:
# load stadium_data as a json object
stadium_dataobj = json.loads(stadium_data)
# print the city of Luzhniki Stadium by navigating through the nested dictionaries and lists
print(stadium_dataobj['stadiums'][1]['city'])

Moscow


In [7]:
# read world cup match data from the json file into a json object
a = open('worldcup.json')
worldcup = json.load(a)
# print the day of the month of the first match
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
print(worldcup['rounds'][0]['matches'][0]['date'][8:10])

14


In [126]:
# read world cup team data from the json file into json object
b = open('worldcup.teams.json')
worldcupteam = json.load(b)
# print the name of the first team in the team data set
# HINT: Once the data is in python data structures (i.e. lists/dictionaries) 
# navigate through the nested lists and dictionaries to extract the relevant information
print(worldcupteam['teams'][0]['name'])

Egypt


In [127]:
# manually create a python json object with soccer associations and continents using dictionaries and lists
# Union of European Football Associations - Europe
# Asian Football Confederation - Asia
# Confederation Africaine de Football - Africa
soccer = [{"Soccer Associations":"Union of European Football Associations",
          "continents":"Europe"},
         {"Soccer Associations":"Asian Football Confederation",
          "continents":"Asia"},
         {"Soccer Associations":"Confederation Africaine de Football",
          "continents":"Africa"}]
# save the soccer associations as a json string
#HINT: json.dumps
soccerstr  = json.dumps(soccer)

# Part B: Flattening json

In [128]:
# flatten the player data set into a data frame with one row per player
players_norm = pd.json_normalize(playersobj,meta="name")
# print first few rows of data set
players_norm.head()

,name,team
0,Gazinsky,Russia
1,Dzyuba,Russia
2,Lukaku,Belgium


In [129]:
# flatten the stadium data set into a data frame with one row per stadium
stadium_norm = pd.json_normalize(stadium_dataobj,record_path="stadiums")
# print first few rows of data set
stadium_norm.head()

,name,city
0,Ekaterinburg Arena,Ekaterinburg
1,Luzhniki Stadium,Moscow
2,Nizhny Novgorod Stadium,Nizhny Novgorod


In [175]:
# flatten the match data set into a data frame with one row per match
# HINT: In order to include more than 2 levels of nested dictionaries in record path
# you have to include them as nested keys (see https://stackoverflow.com/questions/47242845/pandas-io-json-json-normalize-with-very-nested-json)
# e.g. record_path=["level1key",["level2key"]]
# e.g. {'level1key':{'level2key':['my', 'list', 'of', 'interest']}}
worldcup_norm = pd.json_normalize(worldcup,record_path=["rounds",["matches"]])
worldcup_norm.rename(columns = {'team2.name':'team2name'}, inplace=True)
# print number of rows
worldcup_norm.shape[0]
# print first few rows of data set
worldcup_norm.head()

,num,date,time,score1,score2,score1i,score2i,goals1,goals2,group,...,team1.code,team2name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,1,2018-06-14,18:00,5,0,2,0,"[{'name': 'Gazinsky', 'minute': 12, 'score1': ...",[],Group A,...,RUS,Saudi Arabia,KSA,luzhniki,Luzhniki Stadium,NaN,NaN,NaN,NaN,NaN
1,2,2018-06-15,17:00,0,1,0,0,[],"[{'name': 'Giménez', 'minute': 89, 'score1': 0...",Group A,...,EGY,Uruguay,URU,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN
2,3,2018-06-15,21:00,3,3,2,1,"[{'name': 'Ronaldo', 'minute': 4, 'score1': 1,...","[{'name': 'Costa', 'minute': 24, 'score1': 1, ...",Group B,...,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
3,4,2018-06-15,18:00,0,1,0,0,[],"[{'name': 'Bouhaddouz', 'minute': 90, 'offset'...",Group B,...,MAR,Iran,IRN,saintpetersburg,Saint Petersburg Stadium,NaN,NaN,NaN,NaN,NaN
4,5,2018-06-16,13:00,2,1,0,0,"[{'name': 'Griezmann', 'minute': 58, 'score1':...","[{'name': 'Jedinak', 'minute': 62, 'score1': 1...",Group C,...,FRA,Australia,AUS,kazan,Kazan Arena,NaN,NaN,NaN,NaN,NaN


In [131]:
# flatten team data set into a data frame with one row per team
worldcupteam_norm = pd.json_normalize(worldcupteam,record_path=["teams"])
# print number of rows
worldcupteam_norm.shape[0]
# print first few rows of data set
worldcupteam_norm.head()

,name,code,continent,assoc.key,assoc.name,assoc.continental.name,assoc.continental.code
0,Egypt,EGY,Africa,egy,Egyptian Football Association,Confédération Africaine de Football (CAF),CAF
1,Morocco,MAR,Africa,mar,Fédération Royale Marocaine de Football,Confédération Africaine de Football (CAF),CAF
2,Tunisia,TUN,Africa,tun,Fédération Tunisienne de Football,Confédération Africaine de Football (CAF),CAF
3,Senegal,SEN,Africa,sen,Fédération Sénégalaise de Football,Confédération Africaine de Football (CAF),CAF
4,Nigeria,NGA,Africa,nga,Nigeria Football Federation,Confédération Africaine de Football (CAF),CAF


In [132]:
# flatten the association data set into a data frame with one row per association
soccer_norm = pd.json_normalize(soccer)
# print number of rows
soccer_norm.shape[0]
# print first few rows of data set
soccer_norm.head()

,Soccer Associations,continents
0,Union of European Football Associations,Europe
1,Asian Football Confederation,Asia
2,Confederation Africaine de Football,Africa


# Part C: XML

In [133]:
# manually create a string with players information stored as an xml
players_str = '<player><name>Gazinsky</name><team>Russia</team></player><player><name>Dzyuba</name><team>Russia</team></player><player><name>Lukaku</name><team>Belgium</team></player>'
json.dumps(players_str)

'"<player><name>Gazinsky</name><team>Russia</team></player><player><name>Dzyuba</name><team>Russia</team></player><player><name>Lukaku</name><team>Belgium</team></player>"'

In [134]:
# manually create a string with stadium information stored as an xml
stadium_str = '<stadium><name>Ekaterinburg Arena</name><city>Ekaterinburg</city></stadium><stadium><name>Luzhniki Stadium</name><city>Moscow</city></stadium><stadium><name>Nizhny Novgorod Stadium</name><city>Nizhny Novgorod</city></stadium>'
json.dumps(stadium_str)

'"<stadium><name>Ekaterinburg Arena</name><city>Ekaterinburg</city></stadium><stadium><name>Luzhniki Stadium</name><city>Moscow</city></stadium><stadium><name>Nizhny Novgorod Stadium</name><city>Nizhny Novgorod</city></stadium>"'

In [135]:
# manually create a string with an XML DTD corresponding to the players XML
players_dtd = '<?xml version="1.0"><!DOCTYPE players [<!ELEMENT players (player)><!ELEMENT player (name,team)><!ELEMENT name (Gazinsky,Dzyuba,Lukaku)><!ELEMENT city (Russia,Russia,Belgium)>]>'

In [136]:
# manually create a string with an XML DTD corresponding to the stadiums XML
stadium_dtd = '<?xml version="1.0"><!DOCTYPE stadiums [<!ELEMENT stadiums (stadium)><!ELEMENT stadium (name, city)><!ELEMENT name (Ekaterinburg Arena,Luzhniki Stadium,Nizhny Novgorod Stadium)><!ELEMENT city (Ekaterinburg, Moscow,Nizhny Novgorod)>]>'

In [137]:
# read in file ebay.xml as an element tree
ebay_etree = etree.parse('ebay.xml')

# print element tree object
# HINT: It should say ElementTree object at ...
print(ebay_etree)


In [138]:
# view main contents by looping over the main elements of the element tree
# Step 1: search for all listing elements
# Step 2: loop over listing elements
# Step 3: for each listing element get the children
# Step 4: print the tag name of each child
# Step 5: print the content of each child
x = []
for i in ebay_etree.findall('listing'):
    list_elem = ebay_etree.findall('listing')
    x.append(list_elem)
    children = i.getchildren()
    for y in children:
        print(y.tag)
        print(y.text)

seller_info

       
payment_types
Visa/MasterCard, Money Order/Cashiers Checks, Personal Checks, See item description for payment methods accepted
   
shipping_info
Buyer pays fixed shipping charges, Will ship to United States only
   
buyer_protection_info

   
auction_info

     
bid_history

       
item_info

      
seller_info

       
payment_types
Visa/MasterCard, Discover, Money Order/Cashiers Checks, Personal Checks, See item description for payment methods accepted
   
shipping_info
Buyer pays actual shipping charges, Will ship to United States and the following regions: Canada, See item description for shipping charges
   
buyer_protection_info

   
auction_info

     
bid_history

       
item_info

      
seller_info

       
payment_types
Money Order/Cashiers Checks, Personal Checks, See item description for payment methods accepted
   
shipping_info
Buyer pays actual shipping charges, Will ship to United States and the following regions: Canada, See item description for

# Part D: SQL

In [176]:
# create a database and sql connection 
db = sqlite3.connect("database.sql")

In [192]:
# save the players data set as a sql table in your database
players_norm.to_sql('players',db, if_exists ='replace')

In [193]:
# save the stadiums data set as a sql table in your database
stadium_norm.to_sql('stadiums',db, if_exists ='replace')

In [178]:
# save the match data set as a sql table in your database
# HINT: You'll need to flatten stadium, team1, team2 and you can ignore goals1, goals2 
# (you'll need to slice the data frame by selecting only the columns you want to remove goals1, goals2)
# HINT: I suggest using apply combined with lambda function to do this
# e.g. match_df['team1'] = match_df['team1'].apply(lambda x:x['name'])
#worldcup_norm['team1'] = worldcup_norm['team1'].apply(lambda x:x['name'])
#worldcup_norm['team2'] = worldcup_norm['team2'].apply(lambda x:x['name'])
match = worldcup_norm.copy()
matches = match.drop(["goals1", "goals2"], axis = 1)
matches.to_sql('matches',db, if_exists ='replace')

In [194]:
# save the teams data set as a sql table
# HINT: You can remove the association variable
worldcupteam_norm.to_sql('teams',db, if_exists ='replace')

In [187]:
#cursor = db.cursor()
#cursor.execute("DROP table IF EXISTS matches")
matches.head()
matches.shape[0]

64

In [189]:
# select only those rows that Mexico played from the database and save these rows as a data frame
# HINT: You'll need to use WHERE to select the right rows
mexicoquery = """SELECT *
        FROM matches
        WHERE team2name = 'Mexico'"""
mexico = pd.read_sql_query(mexicoquery,db)
# count the number of rows in the data frame
mexico.shape[0]

# print the first few rows of the data frame
mexico.head()

,index,num,date,time,score1,score2,score1i,score2i,group,city,...,team1.code,team2name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,10,11,2018-06-17,18:00,0,1,0,1,Group F,Moscow,...,GER,Mexico,MEX,luzhniki,Luzhniki Stadium,None,None,None,None,NaN
1,27,28,2018-06-23,18:00,1,2,0,1,Group F,Rostov-on-Don,...,KOR,Mexico,MEX,rostov,Rostov Arena,None,None,None,None,NaN
2,52,53,2018-07-02,18:00,2,0,0,0,None,Samara,...,BRA,Mexico,MEX,samara,Samara Arena,None,None,None,None,1.0


In [191]:
# select only those rows that were ties from the database and save these rows as a data frame
# HINT: By tie it means that score1 is the same as score2
tiequery = """SELECT * 
        FROM matches
        WHERE score1 = score2"""
tie = pd.read_sql_query(tiequery,db)

# count the number of rows in the data frame
tie.shape[0]

# print the first few rows of the data frame
tie.head()

,index,num,date,time,score1,score2,score1i,score2i,group,city,...,team1.code,team2name,team2.code,stadium.key,stadium.name,score1et,score2et,score1p,score2p,knockout
0,2,3,2018-06-15,21:00,3,3,2,1,Group B,Sochi,...,POR,Spain,ESP,fisht,Fisht Stadium,NaN,NaN,NaN,NaN,NaN
1,6,7,2018-06-16,16:00,1,1,1,1,Group D,Moscow,...,ARG,Iceland,ISL,spartak,Spartak Stadium,NaN,NaN,NaN,NaN,NaN
2,8,9,2018-06-17,21:00,1,1,1,0,Group E,Rostov-on-Don,...,BRA,Switzerland,SUI,rostov,Rostov Arena,NaN,NaN,NaN,NaN,NaN
3,21,22,2018-06-21,16:00,1,1,1,1,Group C,Samara,...,DEN,Australia,AUS,samara,Samara Arena,NaN,NaN,NaN,NaN,NaN
4,31,32,2018-06-24,20:00,2,2,1,1,Group H,Ekaterinburg,...,JPN,Senegal,SEN,ekaterinburg,Ekaterinburg Arena,NaN,NaN,NaN,NaN,NaN
